In [13]:
import numpy as np
import pandas as pd
import regex as re
from bs4 import BeautifulSoup
import contractions
import unidecode
import re
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
import string
from sklearn.utils import shuffle
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification,RobertaForSequenceClassification,RobertaTokenizer
from transformers import EarlyStoppingCallback
# from gensim.parsing.preprocessing import remove_stopwords
from transformers import XLMRobertaModel, XLMRobertaTokenizer
import os


In [14]:

def removeHtml(sentence):
    soup = BeautifulSoup(sentence, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def removePuncu(sentence):
    res = re.sub(r'[^\w\s]', '', sentence)
    return res

def removeLink(sentence):
    result = re.sub(r'http\S+', '', sentence)
    return result

def removeWhiteSpaces(sentence):
    sentence = sentence.strip()
    return " ".join(sentence.split())

def expand_contractions(sentence):
    sentence = contractions.fix(sentence)
    return sentence

def removeAccented(sentence):
    sentence = unidecode.unidecode(sentence)
    return sentence

def removeSpec(sentence):
    sentence = re.sub("[^A-Z]", " ", sentence, 0, re.IGNORECASE)
    return sentence

def removeNum(sentence):
    pattern = r'[0-9]'
    sentence = re.sub(pattern, '', sentence)
    return sentence

def lowerCase(sentence):
    sentence = sentence.lower()
    return sentenceMergedData/testDataMerged.tsv(\s*)user(\s*)')
#     sentence = remove.sub(" ", sentence)
#     return sentence

# with open(r'C:\Users\LEGION\Documents\Untitled Folder 2\Emoji_Dict.p', 'rb') as fp:
#     Emoji_Dict = pickle.load(fp)
# Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

# def convertEmoji(sentence):
#     for emot in Emoji_Dict:
#         sentence = re.sub(r'('+emot+')', "".join(Emoji_Dict[emot].replace(",","").replace(":","").split()),sentence)
#     return sentence

def add_labels(value):
    if value == 'positive':
        return 2
    elif value == 'negative':
        return 0
    elif value == 'neutral':
        return 1
    

In [15]:
def preprocessing(data):
    sent_or_word = 'translation_sentence_level'
    data[sent_or_word] = data[sent_or_word].apply(lambda x: lowerCase(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removePuncu(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeSpec(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeHtml(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeAccented(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeLink(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeNum(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: expand_contractions(x))
#     da[sent_or_word] = da[sent_or_word].apply(lambda x: convertEmoji(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeWhiteSpaces(x))
    # data[sent_or_word] = data[sent_or_word].apply(lambda x: removeUser(x))
    x = data[sent_or_word].to_list()
    y = list(data['categorical_labels']) 
    
    return x,y


In [16]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [17]:
model_results_dict = {"Model_Name":[], "Accuracy": [], 'Recall': [], 'Precision': [], "f1":[]}


In [18]:
max_length = 128
num_labels = 3
# task = 'sentiment'
model_name = "roberta-base"
# model_name = f"cardiffnlp/twitter-roberta-base-{task}"
# safe_model_name = sanitize(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels = num_labels, ignore_mismatched_sizes = True)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/ablstation2/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 1,

In [19]:
train_data = pd.read_csv('emojiRemovedFiles/MergedEmojiRemoved/trainDataMerged.tsv', sep = '\t')
val_data = pd.read_csv('emojiRemovedFiles/MergedEmojiRemoved/valDataMerged.tsv', sep = '\t')
train_data = train_data[train_data['translation_sentence_level'].notna()]
val_data = val_data[val_data['translation_sentence_level'].notna()]

train_data['categorical_labels'] = train_data['label'].map(add_labels)
val_data['categorical_labels'] = val_data['label'].map(add_labels)


In [20]:
x_train,y_train = preprocessing(train_data)

#splitting the trainData into train test, for checking the performance of our model. 
x_train_final, x_test_final, y_train_final, y_test_final = train_test_split(x_train, y_train, test_size=0.25, random_state=42, shuffle = True)

x_val, y_val = preprocessing(val_data)

x_train_tokenized = tokenizer(x_train_final, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)
x_val_tokenized = tokenizer(x_val, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)

train_dataset = Dataset(x_train_tokenized, y_train_final)
val_dataset = Dataset(x_val_tokenized, y_val)



In [21]:
def compute_metrics(p):
        pred, labels = p
        pred = np.argmax(pred, axis=1)
        accuracy = accuracy_score(y_true=labels, y_pred=pred)
        recall = recall_score(y_true=labels, y_pred=pred,average='weighted')
        precision = precision_score(y_true=labels, y_pred=pred,average='weighted')
        f1 = f1_score(y_true=labels, y_pred=pred,average='weighted')

        return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [22]:
newModelNameForSaving = model_name.split("/")[-1]
x = "Train_merged_Roberta_Removed_Emoji.tsv"

In [23]:
args = TrainingArguments(
    output_dir="checkpointsMergedRobertaRemoveEmoji/{}_{}".format(newModelNameForSaving,x.split(".")[0]),
    evaluation_strategy="epoch",
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    num_train_epochs=5,
    seed=7,
    load_best_model_at_end=True,
    save_strategy='epoch',
    overwrite_output_dir=True
      #save_strategy = "no"
  )

trainer = Trainer(
      model=model,
      args=args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,
      compute_metrics=compute_metrics,
      callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
  )


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
100%|██████████| 26/26 [10:28<00:00, 24.17s/it]


In [24]:
trainer.train()
trainer.save_model("modelMergedRemoveEmojiRoberta/{}_{}".format(newModelNameForSaving,x.split(".")[0]))

# test_data = pd.read_csv("/content/ts_translated_test.tsv",sep='\t')
# test_data['categorical_labels'] = test_data['label'].map(add_labels)

X_test_tokenized = tokenizer(x_test_final, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)


test_dataset = Dataset(X_test_tokenized, y_test_final)


predictions = trainer.predict(test_dataset)

pred = np.argmax(predictions.predictions, axis=-1)

  
accuracy = accuracy_score(y_true=y_test_final, y_pred=pred)
recall = recall_score(y_true=y_test_final, y_pred=pred,average='weighted')
precision = precision_score(y_true=y_test_final, y_pred=pred,average='weighted')
f1 = f1_score(y_true=y_test_final, y_pred=pred,average='weighted')

print(model_name)
print("accuracy: {}".format(accuracy))
print("recall: {}".format(recall))
print("precision: {}".format(precision))
print("f1: {}".format(f1))


model_results_dict['Model_Name'].append(x)
model_results_dict['Accuracy'].append(accuracy)
model_results_dict['Recall'].append(recall)
model_results_dict['Precision'].append(precision)
model_results_dict['f1'].append(f1)

df = pd.DataFrame(data = model_results_dict)
df.to_csv("{}_RemovedEmojiMergedDataRobertaresults.csv".format(newModelNameForSaving))


/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 39514
  Num Epochs = 5
  Instantaneous batch size per device = 256
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 1
  Total optimization steps = 390
  0%|          | 0/390 [00:00<?, ?it/s]/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 20%|██        | 78/390 [00:56<03:03,  1.70it/s]***** Running Evaluat

{'eval_loss': 0.7919630408287048, 'eval_accuracy': 0.6518847006651884, 'eval_precision': 0.6601628453594729, 'eval_recall': 0.6518847006651884, 'eval_f1': 0.6496577249745378, 'eval_runtime': 3.2581, 'eval_samples_per_second': 1799.489, 'eval_steps_per_second': 3.683, 'epoch': 1.0}


Model weights saved in checkpointsMergedRobertaRemoveEmoji/roberta-base_Train_merged_Roberta_Removed_Emoji/checkpoint-78/pytorch_model.bin
/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 40%|████      | 156/390 [01:58<02:16,  1.71it/s]***** Running Evaluation *****
  Num examples = 5863
  Batch size = 256
                                                 
 40%|████      | 156/390 [02:02<02:16,  1.71it/s]Saving model checkpoint to checkpointsMergedRobertaRemoveEmoji/roberta-base_Train_merged_Roberta_Removed_Emoji/checkpoint-156
Configuration saved in checkpointsMergedRobertaRemoveEmoji/roberta-base_Train_merged_Roberta_Removed_Emoji/checkpoint-156/config.json


{'eval_loss': 0.8035369515419006, 'eval_accuracy': 0.6433566433566433, 'eval_precision': 0.6709040312949847, 'eval_recall': 0.6433566433566433, 'eval_f1': 0.6350763319922383, 'eval_runtime': 3.3258, 'eval_samples_per_second': 1762.901, 'eval_steps_per_second': 3.608, 'epoch': 2.0}


Model weights saved in checkpointsMergedRobertaRemoveEmoji/roberta-base_Train_merged_Roberta_Removed_Emoji/checkpoint-156/pytorch_model.bin
/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
 60%|██████    | 234/390 [03:01<01:30,  1.72it/s]***** Running Evaluation *****
  Num examples = 5863
  Batch size = 256
                                                 
 60%|██████    | 234/390 [03:04<01:30,  1.72it/s]Saving model checkpoint to checkpointsMergedRobertaRemoveEmoji/roberta-base_Train_merged_Roberta_Removed_Emoji/checkpoint-234
Configuration saved in checkpointsMergedRobertaRemoveEmoji/roberta-base_Train_merged_Roberta_Removed_Emoji/checkpoint-234/config.json


...

{'eval_loss': 0.8108118176460266, 'eval_accuracy': 0.6595599522428791, 'eval_precision': 0.660352656595419, 'eval_recall': 0.6595599522428791, 'eval_f1': 0.6589347612884077, 'eval_runtime': 3.2896, 'eval_samples_per_second': 1782.308, 'eval_steps_per_second': 3.648, 'epoch': 4.0}


Model weights saved in checkpointsMergedRobertaRemoveEmoji/roberta-base_Train_merged_Roberta_Removed_Emoji/checkpoint-312/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from checkpointsMergedRobertaRemoveEmoji/roberta-base_Train_merged_Roberta_Removed_Emoji/checkpoint-78 (score: 0.7919630408287048).
 80%|████████  | 312/390 [04:09<01:02,  1.25it/s]
Saving model checkpoint to modelMergedRemoveEmojiRoberta/roberta-base_Train_merged_Roberta_Removed_Emoji
Configuration saved in modelMergedRemoveEmojiRoberta/roberta-base_Train_merged_Roberta_Removed_Emoji/config.json


{'train_runtime': 249.4701, 'train_samples_per_second': 791.959, 'train_steps_per_second': 1.563, 'train_loss': 0.7256035437950721, 'epoch': 4.0}


Model weights saved in modelMergedRemoveEmojiRoberta/roberta-base_Train_merged_Roberta_Removed_Emoji/pytorch_model.bin
***** Running Prediction *****
  Num examples = 13172
  Batch size = 256
/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
100%|██████████| 26/26 [00:07<00:00,  3.98it/s]

roberta-base
accuracy: 0.6487245672638932
recall: 0.6487245672638932
precision: 0.658913727683532
f1: 0.646815938229347


In [1]:
import torch
torch.cuda.empty_cache()

/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
